In [15]:
import pandas as pd
import numpy as np

from scipy.stats import pearsonr, wilcoxon, ttest_ind, probplot, shapiro, chi2_contingency
from statsmodels.sandbox.stats.multicomp import multipletests 
from statsmodels.stats.weightstats import *
from numpy import median
import scipy
import itertools
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\Asya\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['exp']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Анализ результатов АБ-теста

In [16]:
data = pd.read_csv('ab_browser_test.csv')

In [17]:
data.head()

,userID,browser,slot,n_clicks,n_queries,n_nonclk_queries
0,1,Browser #2,exp,23,32,19
1,3,Browser #4,exp,3,4,2
2,5,Browser #4,exp,29,35,16
3,6,Browser #4,control,12,6,0
4,7,Browser #4,exp,54,68,30


Описание данных:
- userID: уникальный идентификатор пользователя
- browser: браузер, который использовал userID
- slot: в каком статусе пользователь участвовал в исследовании (exp = видел измененную страницу, control = видел неизменную страницу)
- n_clicks: количество кликов, которые пользоваль совершил за n_queries
- n_queries: количество запросов, который совершил userID, пользуясь браузером browser
- n_nonclk_queries: количество запросов пользователя, в которых им не было совершено ни одного клика  
Обращаем ваше внимание, что не все люди используют только один браузер, поэтому в столбце userID есть повторяющиеся идентификаторы. В предлагаемых данных уникальным является сочетание userID и browser.

In [18]:
exp = data[data['slot'] == 'exp']
control = data[data['slot'] == 'control']

Посчитайте, насколько в группе exp больше пользовательских кликов по сравнению с группой control в процентах от числа кликов в контрольной группе.

Полученный процент округлите до третьего знака после точки

In [19]:
round(((exp['n_clicks'].sum() / control['n_clicks'].sum()) - 1) * 100, 3)

1.614

Для этого постройте с помощью бутстрепа 95% доверительный интервал для средних значений и медиан количества кликов в каждой из двух групп. Отметьте все верные утверждения.

In [51]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [58]:
np.random.seed(0)
exp_bs = get_bootstrap_samples(exp['n_clicks'].values, 40)
control_bs = get_bootstrap_samples(control['n_clicks'].values, 40)
print("means exp: ", stat_intervals(exp_bs.mean(axis = 1), 0.05))
print("means control: ", stat_intervals(control_bs.mean(axis = 1), 0.05))

means exp:  [11.51597095 11.63371724]
means control:  [11.24526083 11.35598937]


In [59]:
print("median exp: ", stat_intervals(np.median(exp_bs, axis = 1), 0.05))
print("median control: ", stat_intervals(np.median(control_bs, axis = 1), 0.05))

median exp:  [5. 5.]
median control:  [4. 4.]


In [55]:
res = np.median(exp_bs, axis = 1) - np.median(control_bs, axis = 1)
stat_intervals(res, 0.05)

array([1., 1.])

In [56]:
res = exp_bs.mean(axis = 1) - control_bs.mean(axis = 1)
stat_intervals(res, 0.05)

array([0.2178811, 0.3673969])

Получите из данных n_boot_samples псевдовыборок.
По каждой из этих выборок посчитайте среднее и сумму квадратов отклонения от выборочного среднего (control_boot_chi_squared)
Для получившегося вектора средних значений из n_boot_samples постройте q-q plot с помощью scipy.stats.probplot для нормального распределения
Для получившегося вектора сумм квадратов отклонения от выборочного среднего постройте qq-plot с помощью scipy.stats.probplot для хи-квадрат распределения с помощью команды

In [ ]:
np.random.seed(0)
n_boot_samples = 500
get_bootstrap_samples(control['n_clicks'], n_samples)